<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Using-matlab-engine-in-python" data-toc-modified-id="Using-matlab-engine-in-python-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Using matlab engine in python</a></span><ul class="toc-item"><li><span><a href="#The-hard-way!" data-toc-modified-id="The-hard-way!-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>The hard way!</a></span></li><li><span><a href="#The-easy-way!" data-toc-modified-id="The-easy-way!-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>The easy way!</a></span></li></ul></li><li><span><a href="#Video-preprocessing" data-toc-modified-id="Video-preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Video preprocessing</a></span><ul class="toc-item"><li><span><a href="#face-detection-and-croping-the-face" data-toc-modified-id="face-detection-and-croping-the-face-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>face detection and croping the face</a></span></li></ul></li></ul></div>

# Using matlab engine in python

## The hard way! 

Defining the variable in python, setting them into MATLAB workspace and then calling MATLAB 
function inside **eval** function, however the variable defined here should be transformed to matrix sing **cell2mat** function.


In [ ]:
import numpy as np
samples = 1000
x = np.linspace(-10, 10, samples)
y = np.zeros(len(x))
i = 0
for j in range(samples):
    if -1 < x[j] < 1:
        y[j] = 1
    i = i + 1

In [ ]:
eng.workspace['y']=y.tolist()
eng.workspace['x']=x.tolist()
eng.eval("subplot(2,1,1);")
eng.eval("plot(cell2mat(x), cell2mat(y));")
eng.eval("title('Step signal u(t + 1) – u(-t + 1)');")
eng.eval("ylabel('Amplitude');")
eng.eval("xlabel('Sample points [t]');")

eng.eval("subplot(2,1,2);")
eng.eval("plot(abs(fftshift(fft(cell2mat(y)))));")
eng.eval("title('FFT of Step signal u(t + 1) – u(-t + 1)');")
eng.eval("ylabel('DFT Values');")
eng.eval("xlabel('Sample points');")

In [ ]:
# import matlab.engine
# import io
# from IPython.core.magic import register_cell_magic
# ip = get_ipython()

# out = io.StringIO()
# err = io.StringIO()

# # Setup matlab cell magic #
# @register_cell_magic
# def matlab_magic(line,cells):
#     out.truncate(0)
#     out.seek(0)
#     err.truncate(0)
#     err.truncate(0)
# #     print(f"line: {line}")
# #     print(f"cell: {cells}")
#     cells_split = cells.split(";")
#     variables_def = [cell for cell in cells_split if "=" in cell]
#     n_variables = len(variables_def)
#     varaible_names = [var_def.partition('=')[0] for var_def in variables_def]
#     # setting the variables from python to MATLAB workspace
#     for variable in varaible_names:
#         var = '''{line}.workspace['{x}']={x}.tolist()'''
#         ip.run_cell(var.format(line=line, x=variable.strip()))
#     commands = cells_split[n_variables:-1]
#     for command in commands:
#         print(command)
#         raw = '''{line}.eval("{command}", nargout=0, stdout=out, stderr=err)'''
#         print(raw)
#         ip.run_cell(raw.format(line=line, command=command))
# #     print(out.getvalue())
# #     print(err.getvalue())
# eng = matlab.engine.start_matlab()
# %%matlab_magic eng

# y = np.linspace(0,1,10);
# plot(y);


## The easy way!

By using magic function it's possible to use the function and programming convention in MATLAB and employ them directly in python. you need a running MATLAB on your device. The important steps are:  
  
  
      1- running the cell below  
      2- defining MATLAB engine  
      3- assigning the variables and use the function in MATLAB convention.  

In [1]:
import matlab.engine
import io
from IPython.core.magic import register_cell_magic
ip = get_ipython()

out = io.StringIO()
err = io.StringIO()

# Setup matlab cell magic #
@register_cell_magic
def matlab_magic(line,cell):
    out.truncate(0)
    out.seek(0)
    err.truncate(0)
    err.truncate(0)
    raw = '''{line}.eval("""{cell}""", nargout=0, stdout=out, stderr=err)'''
    ip.run_cell(raw.format(line=line, cell=cell))
    print(out.getvalue())
    print(err.getvalue())

ModuleNotFoundError: No module named 'matlab'

In [ ]:
eng = matlab.engine.start_matlab()

In [ ]:
%%matlab_magic eng

x = linspace(-10,10, 100);
y = x.^2;
plot(y);b
hold on;
plot(x.^3);

# Video preprocessing

## face detection and croping the face

In [27]:
from mtcnn import MTCNN
import cv2
import pandas as pd
import os


def bounding_box_check(faces, x, y):
    # check the center
    for face in faces:
        bounding_box = face['box']
        if (bounding_box[1] < 0):
            bounding_box[1] = 0
        if (bounding_box[0] < 0):
            bounding_box[0] = 0
        if (bounding_box[0] - 50 > x
                or bounding_box[0] + bounding_box[2] + 50 < x):
            print('change person from')
            print(bounding_box)
            print('to')
            continue
        if (bounding_box[1] - 50 > y
                or bounding_box[1] + bounding_box[3] + 50 < y):
            print('change person from')
            print(bounding_box)
            print('to')
            continue
        return bounding_box

def face_detect(file, detector, frame_path, x_pos, y_pos):
    name = file.replace('.jpg', '').split('-')
    x = x_pos
    y = y_pos
    img = cv2.imread('%s%s' % (frame_path, file ))
    faces = detector.detect_faces(img)
    # check if detected faces
    if (len(faces) == 0):
        print('no face detect: ' + file)
        return  #no face
    bounding_box = bounding_box_check(faces, x, y)
    if (bounding_box == None):
        print('face is not related to given coord: ' + file)
        return
    print(file, " ", bounding_box)
    print(file, " ", x, y)
    crop_img = img[bounding_box[1]:bounding_box[1] + bounding_box[3],
                   bounding_box[0]:bounding_box[0] + bounding_box[2]]
    crop_img = cv2.resize(crop_img, (160, 160))
    cv2.imwrite('%s/frame_' % output_dir + name[0] + '_' + name[1] + '.jpg',
                crop_img)
    #crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
    #plt.imshow(crop_img)
    #plt.show()




In [24]:
!pwd

/Users/zohre/code/Jackfn96/3DS/notebooks


In [29]:
detector = MTCNN()
output_dir = '../../face_input'
file = 'f-001.jpg'
frame_path = '../../frames/'
# # frame_path = '/Users/zohre/code/zohrea/cocktail_effect/notebooks/frames/'
# # file = '1-01.jpg'
# img = cv2.imread('%s%s' % (frame_path, file ))
# imshow(img)
# faces = detector.detect_faces(img)
# bounding_box = bounding_box_check(faces, x, y)
# print(file, " ", bounding_box)
# print(file, " ", x, y)
# crop_img = img[bounding_box[1]:bounding_box[1] + bounding_box[3],
#                    bounding_box[0]:bounding_box[0] + bounding_box[2]]
# crop_img = cv2.resize(crop_img, (160, 160))
# cv2.imwrite('%s/frame_' % output_dir + name[0] + '_' + name[1] + '.jpg',
#                 crop_img)

face_detect(file, detector, frame_path, x, y)

f-001.jpg   [1043, 122, 150, 183]
f-001.jpg   1094 199


In [31]:
detect_range = (1, 11)

In [32]:
detector = MTCNN()
output_dir = '../../face_input'
frame_path = '../../frames/'
for i in range(detect_range[0], detect_range[1]):
    file_name = "f-%03d.jpg" % (i)
    print(file_name)
    if (not os.path.exists('%s%s' % (frame_path, file_name))):
        print('cannot find input: ' + '%s%s' % (frame_path, file_name))
        continue
    face_detect(file_name, detector, frame_path, x, y)

f-001.jpg
f-001.jpg   [1043, 122, 150, 183]
f-001.jpg   1094 199
f-002.jpg
f-002.jpg   [1057, 112, 142, 178]
f-002.jpg   1094 199
f-003.jpg
no face detect: f-003.jpg
f-004.jpg
no face detect: f-004.jpg
f-005.jpg
no face detect: f-005.jpg
f-006.jpg
no face detect: f-006.jpg
f-007.jpg
f-007.jpg   [1059, 111, 142, 186]
f-007.jpg   1094 199
f-008.jpg
f-008.jpg   [1044, 117, 155, 181]
f-008.jpg   1094 199
f-009.jpg
f-009.jpg   [1034, 105, 160, 204]
f-009.jpg   1094 199
f-010.jpg
f-010.jpg   [1045, 107, 152, 200]
f-010.jpg   1094 199
